In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
#complete methodology table as dataframe
ms_complete = pd.read_csv('methods.csv')

In [9]:
#filter to include only columns 'Tools', 'Year', 'PIM', drop unfilled rows, convert year to int values
ms_filtered = ms_complete[["Tools", "Year", "PIM "]].dropna().reset_index()
ms_filtered.Year = ms_filtered.Year.astype(int)

#get sorted list of tools
a = ms_filtered.Tools.str.split(', ', expand=True).to_numpy()
all_tools = np.unique(a[a != np.array(None)])

#create new database with tools as columns
new_columns = {t:None for t in all_tools}
for k in new_columns:
    new_columns[k] = ms_filtered.Tools.str.split(', ', expand=True).isin([k]).any(axis=1)
ms_tools = pd.concat([ms_filtered.drop('Tools', axis=1), pd.DataFrame.from_dict(new_columns)], axis=1)

In [10]:
ms_tools

,index,Year,PIM,AMESTER,Accelergy,Aladdin,Bochs,BookSim,CACTI,CACTI 6.5,...,Timeloop,Torch,USIMM,VaultSim,Verilog-A,Xilinx,ZSim,gem5,memsim,riscvOVPsim
0,0,2024,PNM,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,2024,PUM,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,2,2024,PUM,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,3,2023,PUM,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,4,2023,PNM,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,105,2014,PUM,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
85,107,2014,PNM,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
86,109,2014,PNM,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
87,110,2013,PUM,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
ms_tools['AMESTER'].sum()

1

In [15]:
ms_poptools = ms_tools.drop(['AMESTER', 'Torch'], axis=1)

In [ ]:
#IDEAS: grouped by approaches, percentages to account for different amounts in groups

In [ ]:
#CATEGORIES
#In-House Simulator
#GPU Simulator: GPGPU-Sim, GPUWattch
#DRAM Simulator: DRAMSim2, 
#Circuit Modeling: Synopsys, Verilog-A
#Full-System Simulators: gem5